In [16]:
import os
import numpy as np
segments = os.getcwd().split('/')[0:-1]
os.path.dirname(os.getcwd())

'/user_data/mmhender/llorg'

In [30]:
groupind = np.arange(0,11,1)
group = [[1,2],[3,4],[5,6], [7], [16, 17], [14, 15], [18,19,20,21,22,23], [8, 9], [10,11], [13], [12], [24,25,0]]

voxel_joined_roi_full = np.array([1,2,3,16,0,24, -1],dtype=int)
np.isin(voxel_joined_roi_full,np.concatenate(group[0:11],axis=0))

array([ True,  True,  True,  True, False, False, False])

In [34]:
import torch
out = torch.load('/user_data/mmhender/llorg/gabor_model_fits/S02/gabor_fwrf_May-04-2021_0312/model_params_allROIs')
print(out.keys())

dict_keys(['lmask', 'fmask', 'tuning_masks', 'ori_tuning_masks', 'cyc_per_stim', 'orients_deg', 'aperture', 'voxel_mask', 'brain_nii_shape', 'image_order', 'voxel_index', 'voxel_roi', 'best_params', 'lambdas', 'best_lambdas', 'val_cc', 'best_losses', 'partition_val_sf', 'partition_val_orient'])


In [39]:
np.shape(out['partition_val_orient'])

(2, 9987)

In [42]:
out['best_losses']

array([42.54773 , 75.515915, 31.85236 , ..., 65.04051 , 59.663685,
       54.58349 ], dtype=float32)

In [26]:
np.concatenate([group[ii][:] for ii in range(11)])
np.concatenate(group, axis=0)

array([ 1,  2,  3,  4,  5,  6,  7, 16, 17, 14, 15, 18, 19, 20, 21, 22, 23,
        8,  9, 10, 11, 13, 12, 24, 25,  0])

In [43]:
import sys
import os
import struct
import time
import numpy as np
import h5py
from scipy.io import loadmat
from scipy.stats import pearsonr
from tqdm import tqdm
import pickle
import math
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
%matplotlib inline

fpX = np.float32

import torch 
sys.path.append('/user_data/mmhender/llorg/fwrf_code_from_osf/')
import src.numpy_utility as pnu
from src.file_utility import save_stuff, flatten_dict, embed_dict
from src.gabor_feature_extractor import Gaborizer
from src.torch_fwrf import get_value, set_value
from src.rf_grid import linspace, logspace
import src.numpy_utility as pnu
from src.file_utility import save_stuff, flatten_dict, embed_dict
from src.load_nsd import image_uncolorize_fn
from src.roi import roi_map, iterate_roi

In [46]:
nsd_root = "/lab_data/tarrlab/common/datasets/NSD/"
stim_root = '/user_data/mmhender/nsd_stimuli/stimuli/nsd/'
mask_root = nsd_root + "nsddata/ppdata/"
beta_root = nsd_root + "nsddata_betas/ppdata/"

subject = 1
saveext = ".png"
savearg = {'format':'png', 'dpi': 120, 'facecolor': None}
timestamp = 'May-03-2021_0134'
# timestamp = time.strftime('%b-%d-%Y_%H%M', time.localtime())
model_name = 'gabor_fwrf'

root_dir = '/user_data/mmhender/nsd-master/'
# root_dir   = os.getcwd() + '/'
net_dir    = root_dir + "net/" 

# input_dir  = '/home/styvesg/repo.data/results/nsd/torch_fwrf_full_brain/S%02d/dnn_fwrf_May-10-2020_1814/'
output_dir = root_dir+"output/S%02d/%s_%s/" % (subject,model_name,timestamp) 

In [47]:
## DEFINE ROIS/VOXELS TO USE HERE ##
from src.file_utility import load_mask_from_nii
from src.roi import roi_map, iterate_roi
roi=None
group_names = ['V1', 'V2', 'V3', 'hV4', 'V3ab', 'LO', 'IPS', 'VO', 'PHC', 'MT', 'MST', 'other']
group = [[1,2],[3,4],[5,6], [7], [16, 17], [14, 15], [18,19,20,21,22,23], [8, 9], [10,11], [13], [12], [24,25,0]]

#voxel_mask_full = load_mask_from_nii(mask_root + "subj%02d/func1pt8mm/brainmask_inflated_1.0.nii"%subject)
# voxel_mask_full = load_mask_from_nii(mask_root + "subj%02d/func1pt8mm/brainmask_vcventral_1.0.nii"%subject)

# note we don't seem to have the vcventral mask  - using full brain mask.
voxel_mask_full = load_mask_from_nii(mask_root + "subj%02d/func1pt8mm/brainmask.nii.gz"%subject)
voxel_roi_full  = load_mask_from_nii(mask_root + "subj%02d/func1pt8mm/roi/prf-visualrois.nii.gz"%subject)
voxel_kast_full = load_mask_from_nii(mask_root + "subj%02d/func1pt8mm/roi/Kastner2015.nii.gz"%(subject))
general_mask_full  = load_mask_from_nii(mask_root + "subj%02d/func1pt8mm/roi/nsdgeneral.nii.gz"%(subject))
ncsnr_full = load_mask_from_nii(beta_root + "subj%02d/func1pt8mm/betas_fithrf_GLMdenoise_RR/ncsnr.nii.gz"%subject)
###
brain_nii_shape = voxel_roi_full.shape
#     print (brain_nii_shape)
###
voxel_roi_mask_full = (voxel_roi_full>0).flatten().astype(bool)
voxel_joined_roi_full = np.copy(voxel_kast_full.flatten())  # load kastner rois
voxel_joined_roi_full[voxel_roi_mask_full] = voxel_roi_full.flatten()[voxel_roi_mask_full] # overwrite with prf rois
# the ROIs defined here are mostly based on pRF mapping - but any voxels which weren't given an ROI definition during the 
# prf mapping, will be given a definition based on Kastner atlas.

###
voxel_mask  = np.nan_to_num(voxel_mask_full).flatten().astype(bool)
# taking out the voxels in "other" to reduce the size a bit
voxel_mask_adj = np.copy(voxel_mask)
voxel_mask_adj[np.isin(voxel_joined_roi_full, group[-1])] = False
voxel_mask_adj[np.isin(voxel_joined_roi_full, [-1])] = False

voxel_mask  = voxel_mask_adj

voxel_idx   = np.arange(len(voxel_mask))[voxel_mask]
voxel_roi   = voxel_joined_roi_full[voxel_mask]
voxel_ncsnr = ncsnr_full.flatten()[voxel_mask]

#     print ('full mask length = %d'%len(voxel_mask))
#     print ('selection length = %d'%np.sum(voxel_mask))
print('\nSizes of all defined ROIs in this subject:')
vox_total = 0
for roi_mask, roi_name in iterate_roi(group, voxel_roi, roi_map, group_name=group_names):
    vox_total = vox_total + np.sum(roi_mask)
    print ("%d \t: %s" % (np.sum(roi_mask), roi_name))

print ("%d \t: Total" % (vox_total))

## DEFINE WHICH VOXEL GROUPS TO ANALYZE HERE #####
voxel_mask_this_roi = np.zeros(np.shape(voxel_mask)).astype('bool')
if roi==None:     
    voxel_mask_this_roi[np.isin(voxel_joined_roi_full,np.concatenate(group[0:11],axis=0))] = True
    roi2print = 'allROIs'        
else:
    groupind = [ii for ii in range(len(group_names)) if group_names[ii]==roi]
    groupind = groupind[0]
    voxel_mask_this_roi[np.isin(voxel_joined_roi_full, group[groupind])] = True
    roi2print = roi

print('\nRunning model for %s, %d voxels\n'%(roi2print, np.sum(voxel_mask_this_roi)))

voxel_idx_this_roi = np.arange(len(voxel_mask_this_roi))[voxel_mask_this_roi]


Sizes of all defined ROIs in this subject:
2392 	: V1
2096 	: V2
1674 	: V3
721 	: hV4
965 	: V3ab
488 	: LO
2155 	: IPS
482 	: VO
382 	: PHC
284 	: MT
55 	: MST
0 	: other
11694 	: Total

Running model for allROIs, 11694 voxels



In [52]:
np.shape(voxel_ncsnr)

(11694,)